# Extract various interesting CME times from 
# COR2 (Stereo) Catalog (Angelos Vourlidas etal, 2017+) of CME

Then compare with the LASCO C2 catalog times to get catalog of images we want to use; they wont match 100% as COR2 is for Stereo A/B which have differnt orbit/viewing of Sun (generally) than SOHO. We'll need to take into account the "solar longitude" to make sure that the CME is one which SOHO could have observed (from L2 Halo orbit on Sun-Earth line).

In [32]:
import pandas as pd

Need_To_Redownload = False

# path to the CSV files (by year) which contain COR2 catalog
COR2_data_path = '../COR2_catalog_20170927'

# name of the LASCO catalog
LASCO_Catalog_Filename = '../lasco_img_catalog_lvl_1.txt'

CME_Catalog_Filename = '../cme_event_catalog.txt'

In [18]:


# find all files in COR2 directory
# and load into memory

from os import listdir
from os.path import isfile, join

files = [f for f in listdir(data_path) if isfile(join(data_path, f))]


df = pd.read_csv(join(data_path, files[0]))
foo = df[df.Comments.str.contains('SB') == True]
foo


,S/C,Event ID,Date,Time,CPA,Width,Vrad,Radial Accel.,Vexp,Expansion Accel.,Cross Id,Type,Sep. Angle,Comments
48,A,2633.3292_0Ac,1/9/14,21:39:00,186,28,343,-22,150,34,2633.4021_0B,F,150,KEY ; SB
49,B,2633.4021_0B,1/9/14,21:39:00,-999,-999,-999,-999,-999,-999,2633.3292_0Ac,F,153,KEY ; SB
52,A,2633.7875_0A,1/10/14,6:54:00,-999,-999,-999,-999,-999,-999,2634.6000_0B,L,150,SB
53,B,2634.6000_0B,1/11/14,2:24:00,-999,-999,-999,-999,-999,-999,2633.7875_0A,F,153,SB
61,B,2634.9021_0B,1/11/14,9:39:00,-999,-999,-999,-999,-999,-999,2634.8917_0A,F,153,SB ; PS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339,B,2870.1208_0B,9/3/14,14:54:00,-999,-999,-999,-999,-999,-999,2870.1743_0A,F,160,SB? ; CS ; OFL
1353,B,2874.3500_0B,9/7/14,20:24:00,-999,-999,-999,-999,-999,-999,2874.8028_0A,F,160,3P ; SB
1381,B,2886.8292_0B,9/20/14,7:54:00,-999,-999,-999,-999,-999,-999,2886.8236_1A,W,161,SB
1385,B,2888.0375_0B,9/21/14,12:54:00,-999,-999,-999,-999,-999,-999,NaN,W,161,SB


In [20]:
# pull the cme catalog to local disk
if Need_To_Redownload:
    import requests
    
    # download LASCO catalog now
    #lasco_hdr_url = 'https://lasco-www.nrl.navy.mil/lz/img_hdr.txt'   # level 0.5 data
    lasco_hdr_url = 'https://lasco-www.nrl.navy.mil/lz/level_1/img_hdr.txt'
    response = requests.get(lasco_hdr_url, allow_redirects=True)
    
    # cache locally
    open(LASCO_Catalog_Filename, 'wb').write(response.content)
    

# load data from LASCO img catalog
lasco_catalog = pd.read_fwf(LASCO_Catalog_Filename, index=False,
                            names=['filename', 'date-obs', 'time-obs','telescope', 'expTime',
                                   'NumCols', 'NumRows', 'StartCol', 'StartRow', 'Filter', 
                                   'Polarizer','Observ.Seq', 'LEB Program', 'junk'])

lasco_catalog
c2_catalog = lasco_catalog[lasco_catalog['telescope'] == 'C2']
#c2_catalog

,filename,date-obs,time-obs,telescope,expTime,NumCols,NumRows,StartCol,StartRow,Filter,Polarizer,Observ.Seq,LEB Program,junk
744,25000161.fts,1996/01/23,17:47:06,C2,74.0,512,512,20,1,Orang,Clear,XZR,0.5253,0
745,25000174.fts,1996/01/23,20:01:08,C2,3.8,512,512,20,1,Orang,Clear,XZR,0.5253,0
746,25000175.fts,1996/01/23,20:06:58,C2,3.8,512,512,20,1,Orang,Clear,XZR,0.5253,0
747,25000185.fts,1996/01/23,21:36:17,C2,3.1,512,512,20,1,Orang,Clear,XZR,0.5253,0
779,25000186.fts,1996/01/24,15:43:29,C2,17.1,1024,1024,20,1,Orang,Clear,XZR,0.5253,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913198,25649100.fts,2017/08/31,22:34:39,C2,25.1,1024,1024,20,1,Orang,Clear,XZ%,1.4520,0
913200,25649101.fts,2017/08/31,23:10:42,C2,25.1,1024,1024,20,1,Orang,Clear,XZ%,1.4490,0
913202,25649102.fts,2017/08/31,23:22:37,C2,25.5,1024,1024,20,1,Orang,Clear,XZ%,1.4460,0
913204,25649103.fts,2017/08/31,23:34:50,C2,25.1,1024,1024,20,1,Orang,Clear,XZ%,1.4430,0


In [31]:
slice1 = c2_catalog[c2_catalog['date-obs']=='2014/01/09']
slice1 = slice1[slice1['time-obs'].str.contains(':37:') == True]
slice1

,filename,date-obs,time-obs,telescope,expTime,NumCols,NumRows,StartCol,StartRow,Filter,Polarizer,Observ.Seq,LEB Program,junk
631523,25488066.fts,2014/01/09,21:37:42,C2,25.1,1024,1024,20,1,Orang,Clear,XZ%,5.5477,0


In [34]:
cdaw_data = pd.read_csv(CME_Catalog_Filename)
cdaw_data = cme_data.rename(columns={'Unnamed: 0':'CDAW_Id'})
cdaw_data

,Date,Time,Central,Width,Linear,2nd,order,speed,Accel,Mass,Kinetic,MPA,Remarks,Duration_C2,Duration_C3,StartTime_C3_Delta
0,1996/01/31,06:52:13,274,47,158,219,99,0,-12.3*,2.5e+14*,3.2e+28*,272,Poor Event; Only 3 points\n,26582.278481,116518.987342,9746.835443
1,1996/02/12,05:47:26,91,53,160,100,211,236,2.0*,1.3e+15,1.6e+29,92,Poor Event\n,26250.000000,115062.500000,9625.000000
2,1996/03/02,04:11:53,88,36,108,95,120,177,1.0*,1.5e+14,8.7e+27,86,Poor Event\n,38888.888889,170462.962963,14259.259259
3,1996/03/07,07:51:56,91,43,60,60,61,75,0.1*,-------,-------,87,Poor Event\n,70000.000000,306833.333333,25666.666667
4,1996/03/09,04:27:49,84,39,93,102,82,0,-1.1*,-------,-------,82,Only 3 points\n,45161.290323,197956.989247,16559.139785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7613,2020/08/29,00:00:05,86,16,281,220,339,478,8.4*,-------,-------,89,Poor Event\n,14946.619217,65516.014235,5480.427046
7614,2020/08/29,15:36:05,284,11,196,227,163,0,-4.8*,-------,-------,282,Poor Event\n,21428.571429,93928.571429,7857.142857
7615,2020/08/29,19:48:05,285,10,218,231,204,79,-2.0*,-------,-------,272,Poor Event\n,19266.055046,84449.541284,7064.220183
7616,2020/08/30,01:25:44,258,42,117,0,213,286,3.5*,-------,-------,250,Poor Event\n,35897.435897,157350.427350,13162.393162


In [38]:
test = cdaw_data[cdaw_data['Date'] == '2014/01/09']
#test = test[test['Time'].str.contains(':37:') == True]
test

,Date,Time,Central,Width,Linear,2nd,order,speed,Accel,Mass,Kinetic,MPA,Remarks,Duration_C2,Duration_C3,StartTime_C3_Delta
5456,2014/01/09,02:00:05,260,166,465,400,533,507,4.0,5.8e+15*,6.3e+30*,231,Partial Halo\n,9032.258065,39591.397849,3311.827957
5457,2014/01/09,12:12:05,291,14,260,215,305,391,4.8*,-------,-------,297,Poor Event\n,16153.846154,70807.692308,5923.076923
5458,2014/01/09,19:24:05,204,118,318,138,528,464,8.1*,5.6e+14,2.8e+29,174,yht_reformat; Poor Event\n,13207.547170,57893.081761,4842.767296
